In [1]:
from sklearn.ensemble import VotingRegressor
import lightgbm as lgb
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
df = pd.read_csv('../../EnefitBigData/merge.csv')
df.drop('Unnamed: 0', inplace=True, axis=1)

In [3]:
time_feat = ['datetime']
cat_feat = ['county', 'is_business', 'product_type', 'is_consumption', 'data_block_id']
num_feat = df.drop(time_feat+cat_feat, axis=1).columns

df_train = df.drop(time_feat+cat_feat[-1:], axis=1)

In [10]:
df_train

,county,is_business,product_type,target,is_consumption,lowest_price_per_mwh,highest_price_per_mwh,eic_count,installed_capacity,euros_per_mwh,...,surface_pressure_h,cloudcover_total_h,cloudcover_low_h,cloudcover_mid_h,cloudcover_high_h,windspeed_10m_h,winddirection_10m_h,shortwave_radiation_h,direct_solar_radiation_h,diffuse_radiation_h
0,0,0,1,0.713,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,1,96.590,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,2,0.000,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,2,17.314,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0,3,2.904,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018347,15,1,0,197.233,1,29.0,34.0,15.0,620.0,-1.29,...,1003.76,22.0,5.4,14.2,28.4,1.394444,161.4,0.0,0.0,0.0
2018348,15,1,1,0.000,0,29.0,34.0,20.0,624.5,-1.29,...,1003.76,22.0,5.4,14.2,28.4,1.394444,161.4,0.0,0.0,0.0
2018349,15,1,1,28.404,1,29.0,34.0,20.0,624.5,-1.29,...,1003.76,22.0,5.4,14.2,28.4,1.394444,161.4,0.0,0.0,0.0
2018350,15,1,3,0.000,0,29.0,34.0,55.0,2188.2,-1.29,...,1003.76,22.0,5.4,14.2,28.4,1.394444,161.4,0.0,0.0,0.0


In [4]:
# Let's build separate models for produciton and consumption of electricity
model_parameters = {
    'learning_rate': 0.05,
    'max_depth': 12
}

# standard technique to use mean predictions from several models with different random_state
model_consumption = VotingRegressor([
    (
        f'consumption_lgb_{i}', 
         lgb.LGBMRegressor(**model_parameters, random_state=i)
    ) for i in range(10)
])
model_production = VotingRegressor([
    (
        f'production_lgb_{i}', 
         lgb.LGBMRegressor(**model_parameters, random_state=i)
    ) for i in range(10)
])


        
mask = df_train['is_consumption'] == 1
model_consumption.fit(
    X=df_train[mask].drop(columns=["target"]),
    y=df_train[mask]["target"]
)

mask = df_train['is_consumption'] == 0
model_production.fit(
    X=df_train[mask].drop(columns=["target"]),
    y=df_train[mask]["target"]
)

with open('model_production2.pkl','wb') as f:
    pickle.dump(model_production,f)
    
with open('model_consumption2.pkl','wb') as f:
    pickle.dump(model_production,f)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.098631 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7826
[LightGBM] [Info] Number of data points in the train set: 1009176, number of used features: 34
[LightGBM] [Info] Start training from score 460.587556
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.094839 seconds.
You can set `force_col_wise=tr

In [6]:
# Let's build separate models for produciton and consumption of electricity

# model_parameters = {
#     'n_estimators': 1000,
#     'learning_rate': 0.05,
#     'colsample_bytree': 0.9,
#     'colsample_bynode': 0.6,
#     'lambda_l1': 3.5,
#     'lambda_l2': 1.5,
#     'max_depth': 12
# }

model_parameters = {
    'learning_rate': 0.05,
    'max_depth': 12
}

# standard technique to use mean predictions from several models with different random_state
model = VotingRegressor([
    (
        f'consumption_lgb_{i}', 
         lgb.LGBMRegressor(**model_parameters, random_state=i)
    ) for i in range(10)
])

model.fit(
    X=df_train.drop(columns=["target"]),
    y=df_train["target"]
)

with open('model2.pkl','wb') as f:
    pickle.dump(model,f)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.257989 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7814
[LightGBM] [Info] Number of data points in the train set: 2018352, number of used features: 35
[LightGBM] [Info] Start training from score 274.783658
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.191540 seconds.
You can set `force_col_wise=tr